In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.MIP.mip_inputs import *
from src.optimization.MIP.mip_model import *
from src.optimization.MIP.mip_outputs import *

2021-04-18 13:47:49.242 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-04-18 13:47:49.243 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
roadmapSelected = 'A'

In [3]:
UI_params = {
    'initial_procurement_budget':350000,
    'budget_rate':0.03,
    'initial_operations_budget':300000,
    'planning_interval':[2022,2037],
    'emissions_baseline': data.emissions2020.sum(),#metric tons
    'emissions_target_pct_of_base':0.40,
    'min_miles_replacement_threshold':150000,#miles
    'min_vehicle_age_replacement_threshold':60,#years
    'objective_weights':{'cost':1,'emissions':0},
}

In [4]:
total_costs = {}
end_emissions = {}

### SA1 

In [5]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-1')#.head(30)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [6]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

Using license file C:\Users\elynch\gurobi.lic


2021-04-18 13:47:53.120 INFO    gurobipy.gurobipy: Using license file C:\Users\elynch\gurobi.lic


Academic license - for non-commercial use only


2021-04-18 13:47:53.121 INFO    gurobipy.gurobipy: Academic license - for non-commercial use only


In [7]:
for v in inputs.vehicle_idx:
    for s in validSchedulesPerVehicle[v]:
        decision = x[v,s].x
        if decision == 1:
            print(inputs.age[v,s,:])

[15. 16.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
[15. 16.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
[15. 16. 17. 18. 19.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
[15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
[15.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
[15.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
[ 0.  1.  

In [8]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

4.05663

In [9]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.3796240262048164

In [10]:
total_costs['SA1'] = cost
end_emissions['SA1'] = emissions

### SA2

In [11]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-2')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [12]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [13]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

3.974

In [14]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.39982517097556286

In [15]:
total_costs['SA2'] = cost
end_emissions['SA2'] = emissions

### SA3

In [16]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-3')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [17]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [18]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

4.11416

In [19]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.399604238110333

In [20]:
total_costs['SA3'] = cost
end_emissions['SA3'] = emissions

### SA4

In [21]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-4')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [22]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [23]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

3.90393

In [24]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.39984676043078904

In [25]:
total_costs['SA4'] = cost
end_emissions['SA4'] = emissions

### SA5

In [26]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-5')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [27]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [28]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

2.644

In [29]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.3974801965776177

In [30]:
total_costs['SA5'] = cost
end_emissions['SA5'] = emissions

### SA6

In [31]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-6')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [32]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [33]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

4.4877

In [34]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.39998797213020393

In [35]:
total_costs['SA6'] = cost
end_emissions['SA6'] = emissions

### SA7

In [36]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-7')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [37]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [38]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

2.62959

In [39]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.3974801965776177

In [40]:
total_costs['SA7'] = cost
end_emissions['SA7'] = emissions

### SA8

In [41]:
data = pd.read_excel('../data/NoCode_SensitivityAnalysisData_10APR21.xlsx',sheet_name='SA-8')
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'
data = data.drop_duplicates('equipmentid')
data = data[data.vehicle_idx<1458]

In [42]:
inputs = MIP_Inputs(data,UI_params)
m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules = carnet(inputs)
outputs = MIP_Outputs(data,UI_params,m,x,vehicles,penalty_budget,penalty_emissions,validSchedulesPerVehicle,validSchedules)
alt_solutions = outputs.get_alternative_solutions()
options = select_alternative_solutions(alt_solutions,outputs)

In [43]:
cost = outputs.get_total_cost('A',roadmap_lookup,options)
cost

4.3875

In [44]:
emissions = outputs.get_final_year_emissions('A',roadmap_lookup,options)/inputs.emissions_baseline
emissions

0.3999989464246245

In [45]:
total_costs['SA8'] = cost
end_emissions['SA8'] = emissions

In [55]:
print('overall cost')
total_costs

overall cost


{'SA1': 4.05663,
 'SA2': 3.974,
 'SA3': 4.11416,
 'SA4': 3.90393,
 'SA5': 2.644,
 'SA6': 4.4877,
 'SA7': 2.62959,
 'SA8': 4.3875}

In [49]:
end_emissions

for i in end_emissions:
    end_emissions[i] = round(end_emissions[i],2)

In [56]:
print('Final year emissions % of baseline')
end_emissions

Final year emissions % of baseline


{'SA1': 0.4,
 'SA2': 0.4,
 'SA3': 0.4,
 'SA4': 0.4,
 'SA5': 0.4,
 'SA6': 0.4,
 'SA7': 0.4,
 'SA8': 0.4}